# State: handmade state machine

> A more realistic implementation

In this example we will build a state machine for a phone call.

Here are its possible **states**:
* Off the hook
* Connecting
* Connected
* On hold
* On the hook

We will also define **triggers** that will force a transition from one state to the other, e.g. dialing a call, which would trigger a transition from the off-the-hook state to the connecting state.

In [1]:
from enum import Enum, auto

class State(Enum):
    OFF_HOOK = auto()
    CONNECTING = auto()
    CONNECTED = auto()
    ON_HOLD = auto()
    ON_HOOK = auto()

class Trigger(Enum):
    CALL_DIALED = auto()
    HUNG_UP = auto()
    CALL_CONNECTED = auto()
    PLACED_ON_HOLD = auto()
    TAKEN_OFF_HOLD = auto()
    LEFT_MESSAGE = auto()

With our states and triggers in place, we now need to map the state transitions; in other words, we need to define the rules by which one state can transition to another. We will create a dictionary using the states as keys and a list of tuples for values, where each tuple contains a trigger and the resulting state.

In [2]:
rules = {
    State.OFF_HOOK: [
        (Trigger.CALL_DIALED, State.CONNECTING)
    ],
    State.CONNECTING: [
        (Trigger.HUNG_UP, State.ON_HOOK),
        (Trigger.CALL_CONNECTED, State.CONNECTED)
    ],
    State.CONNECTED: [
        (Trigger.LEFT_MESSAGE, State.ON_HOOK),
        (Trigger.HUNG_UP, State.ON_HOOK),
        (Trigger.PLACED_ON_HOLD, State.ON_HOLD)
    ],
    State.ON_HOLD: [
        (Trigger.TAKEN_OFF_HOLD, State.CONNECTED),
        (Trigger.HUNG_UP, State.ON_HOOK)
    ]
}

We're only missing 2 pieces of information: the starting state and the exiting state that will determine when the machine is done executing.

> Note that some state machines are infinite, e.g. trading robots in financial markets.

We're modeling a single phone call, so we will have an exit state which will represent when the call is done:

In [3]:
state = State.OFF_HOOK
exit_state = State.ON_HOOK

We now have everything in place to code the actual state machine, which will simply be a loop that checks for the exit state on each iteration and exits once it reaches it.

We will request the user's input in order to figure out which state we should transition to, and keep iterating until the call is done:

In [4]:
while state != exit_state:
    print(f'The phone is currently {state}')

    for i in range(len(rules[state])):
        t = rules[state][i][0]
        print(f'{i}: {t}')

    idx = int(input('Select a trigger:'))
    print(f'User selected {idx}')
    s = rules[state][idx][1]
    state = s

print('We are done using the phone.')

The phone is currently State.OFF_HOOK
0: Trigger.CALL_DIALED
User selected 0
The phone is currently State.CONNECTING
0: Trigger.HUNG_UP
1: Trigger.CALL_CONNECTED
User selected 1
The phone is currently State.CONNECTED
0: Trigger.LEFT_MESSAGE
1: Trigger.HUNG_UP
2: Trigger.PLACED_ON_HOLD
User selected 0
We are done using the phone.


And that's it! This approach allows us to model more complex state machines than the classic way because the transitions are much easier to read and define.